### Libraries

In [2754]:
from pathlib import Path
from pprint import pprint

### Variables

In [2755]:
edl_path = 'reference\z old\Avid-Sequence.edl'
drop_frame = False
video_edits = []
audio_edits = []
# edit_files = []

### Open File

In [2756]:
# Opens EDL and extracts its contents into a single string.
def open_edl(edl_path):
    with open(edl_path, 'r') as edl_file:
        edl_contents = edl_file.read()
    edl_extract(edl_contents)    

In [2757]:
# Breasks EDL into a list of fields for each line
def edl_extract(edl_contents): 
    edl_lines = edl_contents.splitlines()

    edl_extracted = []

    for line in edl_lines:
        columns = line.split()
        edl_extracted.append(columns)
    
    
    simplify_edl(edl_extracted)


### Simplify EDL
Removes the extra columns and data from the EDL.  It also removes any lines where the in/out point match on the same line and no real edit is being made.

In [2758]:
def simplify_edl(edl):
	detect_dropframe(edl)
	simplified_edl = []

	for line in range(2, len(edl)):

		if edl[line][0] != "*" and edl[line][-1] != edl[line][-2]:  #if this doesn't have duplicate timecode for edits
			edits = {}
			dissolve_length = 0

			if edl[line][3] == "D":
				dissolve_length = int(edl[line][4])

			edits = {
				"media_type": edl[line][2][0],
				"edit_type": edl[line][3],
				"dissolve_length": dissolve_length,
				"edit_in" : edl[line][-2],
				"edit_out" : edl[line][-1]
				}

			simplified_edl.append(edits)  #  adds edits to the simplified EDL


		elif edl[line][0] == "*":  # lines with clip note
			edits = {
				"media_type" : edl[line][0],
				"media_file": edl[line][1]
			}

			simplified_edl.append(edits)  #  adds edl  to the simplified EDL 
	
	# pprint(simplified_edl)

	edl_split_video_audio(simplified_edl)	

### Detect Dropframe
Not sure if it's neccisary, but I added the funciton anyway in case we need it.

In [2759]:
def detect_dropframe(edl):
	global drop_frame
     
	if edl[1][0] == "FCM:":
            if edl[1][1] == "DROP":
                drop_frame = True
            else:
                drop_frame = False


### Split Video and Audio Edits

In [2760]:
#  Looks for relevant edit information:  Drop/Non-Drop Frame, Title Name, Edits
def edl_split_video_audio(edl):
    # Clears the edit list for parsing
    global video_edits
    global audio_edits

    edl_video = []
    edl_audio = []

    for line in range(0, len(edl)):
        #  Splits audio and video edits into seperate EDLs
        if edl[line]["media_type"] == "V":
            edl_video.append(edl[line])
            if line+1 < len(edl) and edl[line+1]["media_type"] == "*": 
                edl_video.append(edl[line+1])                

        if edl[line]["media_type"] == "A":
            edl_audio.append(edl[line])
            if line+1 < len(edl) and edl[line+1]["media_type"] == "*":
                edl_audio.append(edl[line+1])


    
    # Parses the edits
    if edl_video:
        video_edits = parse_edits(edl_video)

    if edl_audio:
        audio_edits = parse_edits(edl_audio)

    # print("=========VIDEO=============")
    # pprint(edl_video)
    # print("=========AUDIO=============")
    # pprint(edl_audio)


### Parse Edits
Breaks out each edit out of the EDL.  It seperates the dissolves as their own clip.

In [2761]:
def parse_edits(edl):
    edit_list = []

    for line in range(0, len(edl)):  
        edits = {} 

        #  Audio Edits
        if line+1 < len(edl) and edl[line]["media_type"] == "A":
            #  Audio - Cut In - Cut Out 
                # (only option - no audio dissolves done in vantage  - any edits needs to be done on the media file before exporting to vantage)
            edits = {
                "type": "audio_edit",
                "dissolve_in" : 0,
                "dissolve_out" : 0,
                "edit_in" : edl[line]["edit_in"],
                "edit_out" : edl[line]["edit_out"],
                "media_file" : edl[line+1]["media_file"]            
            }
            edit_list.append(edits)

                

        # #  Video Edits
        if (edl[line]["media_type"] == "V"):

            #  DISSOLVE IN
            if (line+2 < len(edl) and
                edl[line]["edit_type"] == "D" and
                edl[line+1]["media_type"] != "*" and
                edl[line+2]["media_type"] == "*"           
                ): 

                edits = {
                    "type" : "dissolve_in",
                    "dissolve_in" : edl[line]["dissolve_length"],
                    "dissolve_out" : 0,
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+2]["media_file"]            
                }

                edit_list.append(edits)

            # #  DISSOLVE OUT
            if (line+1 < len(edl) and
                edl[line]["edit_type"] == "D" and 
                edl[line+1]["media_type"] == "*"
                ): 
                     
                edits = {
                    "type" : "dissolve_out",
                    "dissolve_in" : 0,
                    "dissolve_out" : int(edl[line]["dissolve_length"]),
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+1]["media_file"]            
                }
                  
                edit_list.append(edits)

            #  CUT IN CUT OUT
            if (line+1 < len(edl) and
                edl[line]["edit_type"] == "C" and 
                edl[line+1]["media_type"] == "*"
                ): 
                     
                edits = {
                    "type" : "cut_in_cut_out",
                    "dissolve_in" : 0,
                    "dissolve_out" : 0,
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+1]["media_file"]            
                }
                  
                edit_list.append(edits)       

    consolodated_edits = consolodate_edits(edit_list)

    
    return consolodated_edits

###  Consolodate Edits
This adds the dissolves back into the the beginning/ending of applicable cuts-only edits.

In [2762]:
def consolodate_edits(edl):
	new_edit_list = []
	
	for line in range(0, len(edl)):
		if edl[line]["type"] == "audio_edit":
			new_edits = {
                    "dissolve_in" : edl[line]["dissolve_in"],
                    "dissolve_out" : edl[line]["dissolve_out"],
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line]["media_file"]            
                }
                  
			new_edit_list.append(new_edits)


		elif edl[line]["type"] == "cut_in_cut_out":
			new_in = edl[line]["edit_in"]
			new_out = edl[line]["edit_out"]
			new_dissolve_in = 0
			new_dissolve_out = 0
			new_edits = {}

			#  Dissolve In - Dissolve Out
			for check in range(0, len(edl)):
				if (edl[check]["type"] == "dissolve_in" and
					edl[line]["edit_in"] == edl[check]["edit_out"] and
					edl[line]["media_file"] == edl[check]["media_file"]
					):

					new_in = edl[check]["edit_in"]
					new_dissolve_in = edl[check]["dissolve_in"]


				if (edl[check]["type"] == "dissolve_out" and
					edl[line]["edit_out"] == edl[check]["edit_in"] and
					edl[line]["media_file"] == edl[check]["media_file"]
					):

					new_out = edl[check]["edit_out"]
					new_dissolve_out = edl[check]["dissolve_out"]

			new_edits = {
                    "dissolve_in" : new_dissolve_in,
                    "dissolve_out" : new_dissolve_out,
                    "edit_in" : new_in,
                    "edit_out" : new_out,
                    "media_file" : edl[line]["media_file"]            
                }
                  
			new_edit_list.append(new_edits)

	return new_edit_list


### Execution 

In [2763]:
open_edl(edl_path)


print("------Video Edits-------")
pprint(video_edits)
print("------Audio Edits-------")
pprint(audio_edits)

------Video Edits-------
[{'dissolve_in': 0,
  'dissolve_out': 0,
  'edit_in': '01:00:20:11',
  'edit_out': '01:00:45:05',
  'media_file': 'CUT-IN-CUT-OUT'},
 {'dissolve_in': 10,
  'dissolve_out': 0,
  'edit_in': '01:00:47:24',
  'edit_out': '01:01:08:16',
  'media_file': 'DISSOLVE-IN-CUT-OUT'},
 {'dissolve_in': 10,
  'dissolve_out': 10,
  'edit_in': '01:01:12:03',
  'edit_out': '01:01:32:00',
  'media_file': 'DISSOLVE-IN-DISSOLVE-OUT'},
 {'dissolve_in': 0,
  'dissolve_out': 10,
  'edit_in': '01:01:35:17',
  'edit_out': '01:02:00:02',
  'media_file': 'CUT-IN-DISSOLVE-OUT'}]
------Audio Edits-------
[{'dissolve_in': 0,
  'dissolve_out': 0,
  'edit_in': '01:01:39:05',
  'edit_out': '01:01:42:22',
  'media_file': '\\HUMANASEP2024\\RAIN0630H'}]
